In [3]:
import numpy as np
import math
import cv2
import matplotlib.pyplot as plt
import time
from MtxTfms.Rotation import Rotation
from MtxTfms.create_test_image import create_test_image

In [96]:
image = cv2.imread('../images/test.jpg')
# image = create_test_image()

theta = 90
theta2 = np.radians(75)
# Create the rotation matrix
t_shear =  np.array([
        [1, 1/np.tan(theta2)],
        [0, 1]
    ])


##################################
t_rotate = np.array([
        [round(np.cos(theta), 3), round(-np.sin(theta), 3)],
        [round(np.sin(theta), 3), round(np.cos(theta), 3)]
    ])
##################################

transform = np.dot(t_rotate, t_shear)


# Calculate the pixel vectors with respect to the origin so we can perform the rotation
def calc_pixel_vector(p, o):
    return [int((p[0] - o[0])), int((p[1] - o[1]))]

# Tranform the pixel vector by theta using the rotation matrix
def calc_transform_vector(pxl_vector):
    # return np.dot(t_shear, pxl_vector)
    return np.dot(transform, pxl_vector)

def shear_image(image, theta2):
    # Calculate the origin of the image
    # determine original image shape - decrement by 1 to align with array indexing
    image_x = np.shape(image)[0]
    image_y = np.shape(image)[1]

    origin_x = math.floor(image_x/2)
    origin_y = math.floor(image_y/2)
    origin = [origin_x, origin_y]

    # Generate a canvas based on new image size
    top_left = np.subtract([0, 0], origin)
    top_right = np.subtract([0, image_y], origin)
    bottom_left = np.subtract([image_x, 0], origin)
    bottom_right = np.subtract([image_x, image_y], origin)

    t_top_left = np.abs(np.dot(transform, top_left).astype(int))
    t_top_right = np.abs(np.dot(transform, top_right).astype(int))
    t_bottom_left = np.abs(np.dot(transform, bottom_left).astype(int))
    t_bottom_right = np.abs(np.dot(transform, bottom_right).astype(int))

    print("Image shape: ", np.shape(image))
    print("Image origin: ", origin)
    print(top_left, top_right, bottom_left, bottom_right)

    print(t_top_left, t_top_right, t_bottom_left, t_bottom_right)

    canvas_x = np.max([t_top_left[0], t_top_right[0], t_bottom_left[0], t_bottom_right[0]])*2
    canvas_y = np.max([t_top_left[1], t_top_right[1], t_bottom_left[1], t_bottom_right[1]])*2
    canvas = np.zeros((canvas_x+1, canvas_y+1, 3))+255

    # Generate new origin to align pixels
    new_origin = [int(canvas_x/2), int(canvas_y/2)]

    print("New Origin: ", new_origin)
    print("Canvas: ", np.shape(canvas))


    for i in range(image_x):
        for j in range(image_y):
            pxl_vals = image[i, j]
            pxl_coords = [i, j]
            pxl_vector = calc_pixel_vector(pxl_coords, origin)
            t_vector = calc_transform_vector(pxl_vector)
            result_vector = t_vector + new_origin
            res_x = int(result_vector[0])
            res_y = int(result_vector[1])
            canvas[res_x, res_y] = pxl_vals
    
    return canvas

result = shear_image(image, theta2)


# Save the image in the output directory
cv2.imwrite("./output/shear.png", result)

# Display the image
# im = cv2.imread('./output/shear.png')
# cv2.imshow('window', im)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Image shape:  (960, 1280, 3)
Image origin:  [480, 640]
[-480 -640] [-480  640] [ 480 -640] [480 640]
[864 295] [433 562] [433 562] [864 295]
New Origin:  [864, 562]
Canvas:  (1729, 1125, 3)


True